##Dataset Split - 5 clients (2 person data each) + 2 person data - test set

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Path to your dataset in Drive
csv_path = "/content/drive/MyDrive/Fatigue_Set/final_feature_label_dataset_normalized_interpolated.csv"

# Load dataset
df = pd.read_csv(csv_path)

# Shuffle once for fairness
df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Split into train/test (80/20 global split)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

# Split train_df into 4 equal parts (clients)
client_dfs = []
split_size = len(train_df) // 4
for i in range(4):
    start = i * split_size
    end = (i + 1) * split_size if i < 3 else len(train_df)  # last gets remainder
    client_dfs.append(train_df.iloc[start:end].reset_index(drop=True))

# Save splits for clients and test set
for i, cdf in enumerate(client_dfs):
    cdf.to_csv(f"client_{i+1}.csv", index=False)

test_df.to_csv("global_test.csv", index=False)

print("✅ Dataset prepared: 4 clients + global test set")

✅ Dataset prepared: 4 clients + global test set


##FL Server Code (One - iteration ONLY)

In [ ]:
%%writefile server.py
import torch
import torch.nn as nn
from client import FMAL_Daf, FatigueSessionDataset, collate
from torch.utils.data import DataLoader
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

class Server:
    def __init__(self, modalities_dim, device):
        self.global_model = FMAL_Daf(modalities_dim, lstm_hidden=32, fusion_dim=64, use_time_and_session=True).to(device)
        self.device = device

    def aggregate(self, client_results):
        """
        client_results: list of tuples -> (state_dict, num_samples)
        Performs Weighted FedAvg aggregation
        """
        client_states, client_sizes = zip(*client_results)
        total_samples = sum(client_sizes)

        # start with zeroed global weights
        new_state = {k: torch.zeros_like(v, device=self.device) for k, v in client_states[0].items()}

        # weighted averaging
        for state, size in zip(client_states, client_sizes):
            weight = size / total_samples
            for k in state.keys():
              if state[k].dtype in (torch.float32, torch.float64):
                  # Only average floating-point tensors (weights, biases)
                  new_state[k] += weight * state[k].to(self.device)
              else:
                  # For integer/buffer values (e.g., num_batches_tracked), just take from one client
                  new_state[k] = state[k].to(self.device)

        self.global_model.load_state_dict(new_state)

    def evaluate_global(self, test_csv):
        df = pd.read_csv(test_csv)
        session_map = {s: i for i, s in enumerate(sorted(df['session'].astype(int).unique()))}
        test_ds = FatigueSessionDataset(df, session_map=session_map, compute_elapsed=True)
        test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=collate)

        self.global_model.eval()
        phys_preds, phys_labels, ment_preds, ment_labels = [], [], [], []
        with torch.no_grad():
            for modals, dom, session_feat, elapsed_feat, (phys, ment) in test_loader:
                modals = [m.to(self.device) for m in modals]
                session_feat = session_feat.to(self.device)
                elapsed_feat = elapsed_feat.to(self.device)
                p_pred, m_pred = self.global_model(modals, dom.to(self.device), session_feat, elapsed_feat)
                phys_preds.extend(p_pred.squeeze().cpu().numpy().tolist())
                ment_preds.extend(m_pred.squeeze().cpu().numpy().tolist())
                phys_labels.extend(phys.squeeze().cpu().numpy().tolist())
                ment_labels.extend(ment.squeeze().cpu().numpy().tolist())

        rmse_phys = math.sqrt(mean_squared_error(phys_labels, phys_preds))
        mae_phys  = mean_absolute_error(phys_labels, phys_preds)
        r2_phys   = r2_score(phys_labels, phys_preds)
        rmse_ment = math.sqrt(mean_squared_error(ment_labels, ment_preds))
        mae_ment  = mean_absolute_error(ment_labels, ment_preds)
        r2_ment   = r2_score(ment_labels, ment_preds)

        print("\n=== Global Test Metrics ===")
        print(f"Physical Fatigue - RMSE: {rmse_phys:.6f}, MAE: {mae_phys:.6f}, R²: {r2_phys:.6f}")
        print(f"Mental Fatigue  - RMSE: {rmse_ment:.6f}, MAE: {mae_ment:.6f}, R²: {r2_ment:.6f}")

Overwriting server.py


##FL Client Code

In [ ]:
%%writefile client.py
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.nn.functional as F
import copy

# === Dataset & Collate ===
class FatigueSessionDataset(Dataset):
    def __init__(self, df, session_map=None, compute_elapsed=True):
        df = df.copy()
        df['session'] = df['session'].astype(int)
        df['person'] = df['person'].astype(int)
        df['window_start'] = pd.to_datetime(df['window_start'])
        if compute_elapsed:
            df['elapsed_time'] = df.groupby(['person','session'])['window_start']\
                                   .transform(lambda x: (x-x.min()).dt.total_seconds())
        else:
            df['elapsed_time'] = 0.0
        self.data = df.sort_values(['person','session','window_start']).reset_index(drop=True)
        if session_map is None:
            self.session_map = {s: i for i, s in enumerate(sorted(self.data['session'].unique()))}
        else:
            self.session_map = session_map
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        HR   = torch.tensor([row['hr_mean'], row['hr_std']], dtype=torch.float32)
        IBI  = torch.tensor([row['duration_mean'], row['duration_std']], dtype=torch.float32)
        ACC  = torch.tensor([row['ax_mean'], row['ax_std'],row['ay_mean'], row['ay_std'],row['az_mean'], row['az_std']], dtype=torch.float32)
        EDA  = torch.tensor([row['eda_mean'], row['eda_std']], dtype=torch.float32)
        Temp = torch.tensor([row['temp_mean'], row['temp_std']], dtype=torch.float32)
        phys = torch.tensor([row['physicalFatigueScore']], dtype=torch.float32)
        ment = torch.tensor([row['mentalFatigueScore']], dtype=torch.float32)
        session_index = self.session_map[int(row['session'])]
        session_feat = torch.tensor([float(session_index)], dtype=torch.float32)
        elapsed = torch.tensor([float(row['elapsed_time'])], dtype=torch.float32)
        return [HR, IBI, ACC, EDA, Temp], torch.tensor([0.0]), session_feat, elapsed, (phys, ment)

def collate(batch):
    modal_inputs = [torch.stack([b[0][i] for b in batch]) for i in range(5)]
    dom  = torch.stack([b[1] for b in batch])
    sess = torch.stack([b[2] for b in batch])
    elap = torch.stack([b[3] for b in batch])
    phys = torch.stack([b[4][0] for b in batch])
    ment = torch.stack([b[4][1] for b in batch])
    return modal_inputs, dom, sess, elap, (phys, ment)

# === Model ===
class ModalityLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=32):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
    def forward(self, x):
        if x.ndim == 2: x = x.unsqueeze(1)
        out,_ = self.lstm(x)
        return out[:,-1,:]

class CrossModalAttention(nn.Module):
    def __init__(self, n_modalities, hidden_dim=32, fusion_dim=64):
        super().__init__()
        self.query = nn.Linear(hidden_dim,fusion_dim)
        self.key   = nn.Linear(hidden_dim,fusion_dim)
        self.value = nn.Linear(hidden_dim,fusion_dim)
    def forward(self, features, domain_disc, debug=False):
        x = torch.stack(features, dim=1)
        Q,K,V = self.query(x), self.key(x), self.value(x)
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (K.size(-1)**0.5)
        attn = F.softmax(scores, dim=-1)
        fused = torch.matmul(attn, V).mean(dim=1)
        return fused

class DomainAdaptiveLayer(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc = nn.Linear(dim, dim); self.bn = nn.BatchNorm1d(dim)
    def forward(self, x): return self.bn(self.fc(x))

class FMAL_Daf(nn.Module):
    def __init__(self, modalities_dim, lstm_hidden=32, fusion_dim=64, num_sessions=3, use_time_and_session=True):
        super().__init__()
        self.use_time_and_session = use_time_and_session
        self.modality_lstms = nn.ModuleList([ModalityLSTM(d,lstm_hidden) for d in modalities_dim])
        self.attn_fusion = CrossModalAttention(len(modalities_dim), lstm_hidden, fusion_dim)
        self.domain_adapt = DomainAdaptiveLayer(fusion_dim)
        self.session_emb = nn.Embedding(num_sessions, 8)
        self.time_enc = nn.Sequential(nn.Linear(1,8), nn.ReLU(), nn.Linear(8,8))
        global_input_dim = fusion_dim+16 if use_time_and_session else fusion_dim
        self.global_lstm = nn.LSTM(global_input_dim, 32, batch_first=True)
        self.reg_phys = nn.Linear(32,1); self.reg_ment = nn.Linear(32,1)
    def forward(self, modal_inputs, domain_disc, session_feat, elapsed_feat):
        feats = [m(inp) for m,inp in zip(self.modality_lstms, modal_inputs)]
        fused = self.attn_fusion(feats, domain_disc)
        adapted = self.domain_adapt(fused)
        if self.use_time_and_session:
            sess_emb = self.session_emb(session_feat.squeeze(-1).long())
            time_emb = self.time_enc(elapsed_feat)
            adapted = torch.cat([adapted, sess_emb, time_emb], dim=1)
        glstm_out,_ = self.global_lstm(adapted.unsqueeze(1))
        feat = glstm_out[:,-1,:]
        return self.reg_phys(feat), self.reg_ment(feat)

# === Client ===
class Client:
    def __init__(self, csv_path, modalities_dim, device):
        df = pd.read_csv(csv_path)
        self.session_map = {s:i for i,s in enumerate(sorted(df['session'].astype(int).unique()))}
        self.dataset = FatigueSessionDataset(df, session_map=self.session_map, compute_elapsed=True)
        self.loader = DataLoader(self.dataset, batch_size=32, shuffle=True, collate_fn=collate)
        self.model = FMAL_Daf(modalities_dim, lstm_hidden=32, fusion_dim=64, use_time_and_session=True).to(device)
        self.device = device

    def train_one_epoch(self, crit, optim):
        self.model.train()
        tot = 0.0
        for modals, dom, sess, elap, (phys,ment) in self.loader:
            modals = [m.to(self.device) for m in modals]
            sess,elap,dom,phys,ment = sess.to(self.device),elap.to(self.device),dom.to(self.device),phys.to(self.device),ment.to(self.device)
            optim.zero_grad()
            p_pred,m_pred = self.model(modals, dom, sess, elap)
            loss = crit(p_pred,phys) + 2*crit(m_pred,ment)   # weight mental fatigue higher
            loss.backward()
            optim.step()
            tot += loss.item()
        return tot/len(self.loader)

    # ✅ NEW: send model weights + dataset size to server
    def get_weights_and_size(self):
        return copy.deepcopy(self.model.state_dict()), len(self.dataset)

Overwriting client.py


##Run Simulation

In [ ]:
import torch
import torch.nn as nn
from client import Client
from server import Server

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# modality dims are [HR(2), IBI(2), ACC(6), EDA(2), Temp(2)]
modalities_dim = [2,2,6,2,2]

# Initialize server + clients
server = Server(modalities_dim, device)
clients = [Client(f"client_{i+1}.csv", modalities_dim, device) for i in range(4)]

crit = nn.MSELoss()

# Federated training loop
rounds = 5
epochs_per_client = 1
for r in range(rounds):
    print(f"\n--- Federated Round {r+1} ---")
    client_results = []
    for i, client in enumerate(clients):
        optim = torch.optim.Adam(client.model.parameters(), lr=1e-3)
        for e in range(epochs_per_client):
            loss = client.train_one_epoch(crit, optim)
            print(f" Client {i+1} Epoch {e+1}: Loss={loss:.4f}")
        client_results.append(client.get_weights_and_size())
    server.aggregate(client_results)

# Evaluate global model
server.evaluate_global("global_test.csv")


--- Federated Round 1 ---
 Client 1 Epoch 1: Loss=0.2115
 Client 2 Epoch 1: Loss=0.4673
 Client 3 Epoch 1: Loss=0.1701
 Client 4 Epoch 1: Loss=0.1924


RuntimeError: result type Float can't be cast to the desired output type Long

#FL Implementation With Novelty

##Install necessary libraries

In [ ]:
!pip install flwr["simulation"] torch pandas scikit-learn -q flwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.6/617.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.3/242.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 

##Code to ignore some deprecation warnings

In [ ]:
# Cell 1 (The very first cell in your notebook)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppresses TensorFlow/XLA warnings

print("TensorFlow logging is configured to be silent.")

TensorFlow logging is configured to be silent.


In [ ]:
# Keep this cell at the top of your notebook (e.g., as Cell 2)
import logging
import warnings

# This silences the SERVER-SIDE INFO logs
logging.getLogger("flwr").setLevel(logging.ERROR)

# This is a good general practice, but the client-side fix is more important
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

print("Logging and warnings configured for a cleaner output.")

Logging and warnings configured for a cleaner output.


##Model Code

In [ ]:
# Replace your existing Cell 4 with this
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from collections import OrderedDict
import flwr as fl
import numpy as np
from typing import Dict, List, Tuple
from flwr.common import Context

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# ========== Dataset with THE WINNING INGREDIENT ==========
class FatigueSessionDataset(Dataset):
    def __init__(self, df, session_map=None, compute_elapsed=True):
        df = df.copy()
        df['session'] = df['session'].astype(int)
        df['person'] = df['person'].astype(int)
        df['window_start'] = pd.to_datetime(df['window_start'])
        if compute_elapsed:
            df['elapsed_time'] = df.groupby(['person', 'session'])['window_start'].transform(lambda x: (x - x.min()).dt.total_seconds())
        else:
            df['elapsed_time'] = 0.0
        self.data = df.sort_values(['person', 'session', 'window_start']).reset_index(drop=True)
        self.session_map = session_map if session_map is not None else {s: i for i, s in enumerate(sorted(df['session'].unique()))}

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx].copy()
        # --- THE WINNING INGREDIENT: On-the-fly Data Augmentation ---
        noise_level = 0.015
        row['hr_mean'] += np.random.normal(0, noise_level)
        row['duration_mean'] += np.random.normal(0, noise_level)
        row['ax_mean'] += np.random.normal(0, noise_level)
        row['ay_mean'] += np.random.normal(0, noise_level)
        row['az_mean'] += np.random.normal(0, noise_level)
        row['eda_mean'] += np.random.normal(0, noise_level)
        row['temp_mean'] += np.random.normal(0, noise_level)
        # --- End of Augmentation ---
        HR = torch.tensor([row['hr_mean'], row['hr_std']], dtype=torch.float32)
        IBI = torch.tensor([row['duration_mean'], row['duration_std']], dtype=torch.float32)
        ACC = torch.tensor([row['ax_mean'], row['ax_std'], row['ay_mean'], row['ay_std'], row['az_mean'], row['az_std']], dtype=torch.float32)
        EDA = torch.tensor([row['eda_mean'], row['eda_std']], dtype=torch.float32)
        Temp = torch.tensor([row['temp_mean'], row['temp_std']], dtype=torch.float32)
        phys = torch.tensor([row['physicalFatigueScore']], dtype=torch.float32)
        ment = torch.tensor([row['mentalFatigueScore']], dtype=torch.float32)
        session_index = self.session_map[int(row['session'])]
        session_feat = torch.tensor([float(session_index)], dtype=torch.float32)
        elapsed = torch.tensor([float(row['elapsed_time'])], dtype=torch.float32)
        domain_disc = torch.tensor([0.0], dtype=torch.float32)
        return [HR, IBI, ACC, EDA, Temp], domain_disc, session_feat, elapsed, (phys, ment)

def collate(batch):
    modal_inputs = [torch.stack([b[0][i] for b in batch]) for i in range(5)]
    domain_disc  = torch.stack([b[1] for b in batch])
    session_feat = torch.stack([b[2] for b in batch])
    elapsed_feat = torch.stack([b[3] for b in batch])
    phys         = torch.stack([b[4][0] for b in batch])
    ment         = torch.stack([b[4][1] for b in batch])
    return modal_inputs, domain_disc, session_feat, elapsed_feat, (phys, ment)

# ========== Model Components (Unchanged) ==========
class ModalityLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=32):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
    def forward(self, x):
        if x.ndim == 2: x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        return out[:, -1, :]

class DomainScalingMLP(nn.Module):
    def __init__(self, input_dim=1, n_modalities=5, hidden=16):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(input_dim, hidden), nn.ReLU(), nn.Linear(hidden, n_modalities), nn.Sigmoid())
    def forward(self, domain_disc): return self.net(domain_disc)

class CrossModalAttention(nn.Module):
    def __init__(self, n_modalities, hidden_dim=32, fusion_dim=64):
        super().__init__()
        self.query, self.key, self.value = nn.Linear(hidden_dim, fusion_dim), nn.Linear(hidden_dim, fusion_dim), nn.Linear(hidden_dim, fusion_dim)
        self.scaler = DomainScalingMLP(input_dim=1, n_modalities=n_modalities, hidden=16)
    def forward(self, features, domain_disc, debug=False):
        x = torch.stack(features, dim=1)
        Q, K, V = self.query(x), self.key(x), self.value(x)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (K.size(--1) ** 0.5)
        attn = F.softmax(scores, dim=-1)
        scaling = self.scaler(domain_disc).unsqueeze(1)
        attn = F.softmax(attn * scaling, dim=-1)
        fused = torch.matmul(attn, V).mean(dim=1)
        return fused

class DomainAdaptiveLayer(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc = nn.Linear(dim, dim)
        self.bn = nn.BatchNorm1d(dim)
    def forward(self, x): return self.bn(self.fc(x))

# ========== Main Model Architecture (Smaller Version) ==========
class FMAL_Daf(nn.Module):
    def __init__(self, modalities_dim, lstm_hidden=32, fusion_dim=64, num_sessions=3, use_time_and_session=True):
        super().__init__()
        self.use_time_and_session = use_time_and_session
        self.modality_lstms = nn.ModuleList([ModalityLSTM(d, lstm_hidden) for d in modalities_dim])
        self.attn_fusion = CrossModalAttention(len(modalities_dim), lstm_hidden, fusion_dim)
        self.domain_adapt = DomainAdaptiveLayer(fusion_dim)
        self.session_emb = nn.Embedding(num_sessions, 8)
        self.time_enc = nn.Sequential(nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 8))
        global_input_dim = fusion_dim + 16 if use_time_and_session else fusion_dim
        self.global_lstm = nn.LSTM(global_input_dim, 32, batch_first=True)
        self.reg_phys = nn.Linear(32, 1)
        self.reg_ment = nn.Linear(32, 1)
        self.apply(self._init_weights)
    @staticmethod
    def _init_weights(m):
        if isinstance(m, (nn.Linear)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None: nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LSTM):
            for name, param in m.named_parameters():
                if 'weight' in name: nn.init.xavier_uniform_(param)
                elif 'bias' in name: nn.init.zeros_(param)
    def forward(self, modal_inputs, domain_disc, session_feat=None, elapsed_feat=None, debug=False):
        feats = [m(inp) for m, inp in zip(self.modality_lstms, modal_inputs)]
        fused = self.attn_fusion(feats, domain_disc, debug=debug)
        adapted = self.domain_adapt(fused)
        if self.use_time_and_session:
            session_emb = self.session_emb(session_feat.squeeze(-1).long())
            time_emb = self.time_enc(elapsed_feat)
            adapted = torch.cat([adapted, session_emb, time_emb], dim=1)
        glstm_out, _ = self.global_lstm(adapted.unsqueeze(1))
        feat = glstm_out[:, -1, :]
        return self.reg_phys(feat), self.reg_ment(feat)

##Train and Test Function

In [ ]:
# Replace your existing Cell 6 with this
def train(net, trainloader, epochs):
    net.train()
    optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)
    criterion = nn.MSELoss()
    for _ in range(epochs):
        for modals, dom, session_feat, elapsed_feat, (phys, ment) in trainloader:
            modals = [m.to(DEVICE) for m in modals]
            dom, session_feat, elapsed_feat = dom.to(DEVICE), session_feat.to(DEVICE), elapsed_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)
            optimizer.zero_grad()
            p_pred, m_pred = net(modals, dom, session_feat, elapsed_feat)
            loss = criterion(p_pred, phys) + 2 * criterion(m_pred, ment)
            loss.backward()
            optimizer.step()

def test(net, testloader):
    net.eval()
    criterion = nn.MSELoss()
    total_loss, all_phys_preds, all_phys_labels, all_ment_preds, all_ment_labels = 0.0, [], [], [], []
    with torch.no_grad():
        for modals, dom, session_feat, elapsed_feat, (phys, ment) in testloader:
            modals = [m.to(DEVICE) for m in modals]
            dom, session_feat, elapsed_feat = dom.to(DEVICE), session_feat.to(DEVICE), elapsed_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)
            p_pred, m_pred = net(modals, dom, session_feat, elapsed_feat)
            total_loss += (criterion(p_pred, phys) + criterion(m_pred, ment)).item()
            all_phys_preds.extend(p_pred.squeeze().cpu().numpy())
            all_phys_labels.extend(phys.squeeze().cpu().numpy())
            all_ment_preds.extend(m_pred.squeeze().cpu().numpy())
            all_ment_labels.extend(ment.squeeze().cpu().numpy())
    loss = total_loss / len(testloader) if len(testloader) > 0 else 0
    rmse_phys = math.sqrt(mean_squared_error(all_phys_labels, all_phys_preds))
    mae_phys = mean_absolute_error(all_phys_labels, all_phys_preds)
    r2_phys = r2_score(all_phys_labels, all_phys_preds)
    rmse_ment = math.sqrt(mean_squared_error(all_ment_labels, all_ment_preds))
    mae_ment = mean_absolute_error(all_ment_labels, all_ment_preds)
    r2_ment = r2_score(all_ment_labels, all_ment_preds)
    metrics = {"loss": loss, "rmse_physical": rmse_phys, "mae_physical": mae_phys, "r2_physical": r2_phys, "rmse_mental": rmse_ment, "mae_mental": mae_ment, "r2_mental": r2_ment}
    return loss, metrics

##Dataset Processing

In [ ]:
# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Update this path ---
CSV_PATH = '/content/drive/MyDrive/Fatigue_Set/final_feature_label_dataset_normalized_interpolated.csv'
NUM_CLIENTS = 10

def load_datasets():
    """Loads the dataset and partitions it for the clients."""
    df = pd.read_csv(CSV_PATH)
    session_map = {s: i for i, s in enumerate(sorted(df['session'].astype(int).unique()))}
    num_sessions = len(session_map)
    full_dataset = FatigueSessionDataset(df, session_map=session_map, compute_elapsed=True)
    len_train = int(len(full_dataset) * 0.8)
    len_val = len(full_dataset) - len_train
    ds_train, ds_val = random_split(full_dataset, [len_train, len_val], torch.Generator().manual_seed(42))
    partition_size = len(ds_train) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    for i in range(len(ds_train) % NUM_CLIENTS): lengths[i] += 1
    datasets = random_split(ds_train, lengths, torch.Generator().manual_seed(42))
    trainloaders, valloaders = [], []
    for ds in datasets:
        trainloaders.append(DataLoader(ds, batch_size=32, shuffle=True, collate_fn=collate, drop_last=True)) # Bug fix
        valloaders.append(DataLoader(ds_val, batch_size=32, shuffle=False, collate_fn=collate))
    testloader = DataLoader(ds_val, batch_size=32, shuffle=False, collate_fn=collate)
    return trainloaders, valloaders, testloader, num_sessions

trainloaders, valloaders, testloader, NUM_SESSIONS = load_datasets()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


##Flower Client Code

In [ ]:
# Replace your existing Cell 7 with this entire block of code

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net, self.trainloader, self.valloader = net, trainloader, valloader
    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]
    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        # This correctly calls the simple 'train' function for your winning FedAvg run
        train(self.net, self.trainloader, epochs=8)
        return self.get_parameters(config={}), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, metrics = test(self.net, self.valloader)

        # --- THIS IS THE FINAL, CRUCIAL FIX ---
        # We now correctly return all 3 items Flower expects: (loss, num_examples, metrics)
        return loss, len(self.valloader.dataset), metrics

def client_fn(cid: str) -> fl.client.Client:
    net = FMAL_Daf([2,2,6,2,2], lstm_hidden=32, fusion_dim=64, num_sessions=NUM_SESSIONS).to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # This updated return statement also fixes the deprecation warnings for a cleaner output
    return FlowerClient(net, trainloader, valloader).to_client()

##Server strategy code

In [ ]:
# Replace your existing Cell 8 with this
from flwr.server.strategy import FedAvg
from flwr.common import ndarrays_to_parameters

def get_evaluate_fn(testloader, num_sessions):
    def evaluate(server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]):
        net = FMAL_Daf([2,2,6,2,2], lstm_hidden=32, fusion_dim=64, num_sessions=num_sessions).to(DEVICE)
        params_dict = zip(net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        net.load_state_dict(state_dict, strict=True)
        loss, metrics = test(net, testloader)
        print("\n" + "="*50)
        print(f"Server-side evaluation round {server_round}")
        print(f"  - Loss: {metrics['loss']:.4f}")
        print(f"  - Physical Fatigue:")
        print(f"    - RMSE: {metrics['rmse_physical']:.4f} | MAE: {metrics['mae_physical']:.4f} | R² Score: {metrics['r2_physical']:.4f}")
        print(f"  - Mental Fatigue:")
        print(f"    - RMSE: {metrics['rmse_mental']:.4f} | MAE: {metrics['mae_mental']:.4f} | R² Score: {metrics['r2_mental']:.4f}")
        print("="*50 + "\n")
        return loss, metrics
    return evaluate

net = FMAL_Daf([2,2,6,2,2], lstm_hidden=32, fusion_dim=64, num_sessions=NUM_SESSIONS).to(DEVICE)
initial_parameters = ndarrays_to_parameters([val.cpu().numpy() for _, val in net.state_dict().items()])

strategy = FedAvg(
    fraction_fit=1.0, fraction_evaluate=1.0, min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS, min_available_clients=NUM_CLIENTS,
    evaluate_fn=get_evaluate_fn(testloader, NUM_SESSIONS),
    initial_parameters=initial_parameters,
    on_evaluate_config_fn=lambda r: {"round": r},
)

##Start Simulation

In [ ]:
# Start the simulation for 80 rounds
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=80),
    strategy=strategy,
)

print("Federated Learning Championship Run finished.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=80, no round_timeout
2025-09-07 07:43:11,094	INFO worker.py:1771 -- Started a local Ray instance.
/usr/local/lib/python3.12/dist-packages/jupyter_clien


Server-side evaluation round 0
  - Loss: 1.0614
  - Physical Fatigue:
    - RMSE: 0.9352 | MAE: 0.9028 | R² Score: -17.8319
  - Mental Fatigue:
    - RMSE: 0.4315 | MAE: 0.3612 | R² Score: -2.2457



(pid=2706) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=2706) E0000 00:00:1757231008.852595    2706 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=2706) E0000 00:00:1757231008.865515    2706 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=2704) W0000 00:00:1757231008.881657    2704 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(pid=2704) W0000 00:00:1757231008.881723    2704 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(pid=2704) W0000 00:00:1757231008.881730    2704 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking t


Server-side evaluation round 1
  - Loss: 0.1025
  - Physical Fatigue:
    - RMSE: 0.2183 | MAE: 0.1813 | R² Score: -0.0262
  - Mental Fatigue:
    - RMSE: 0.2349 | MAE: 0.1863 | R² Score: 0.0377



(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2704) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 4x across cluster]
(ClientAppActor pid=2704)             This is a deprecated feature. It will be removed [repeated 4x across cluster]
(ClientAppActor pid=2704)             entirely in future versions of Flower. [repeated 4x across cluster]
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppAct

...

(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2706) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 10x across cluster]
(ClientAppActor pid=2706)             This is a deprecated feature. It will be removed [repeated 10x across cluster]
(ClientAppActor pid=


Server-side evaluation round 80
  - Loss: 0.0376
  - Physical Fatigue:
    - RMSE: 0.1319 | MAE: 0.1002 | R² Score: 0.6257
  - Mental Fatigue:
    - RMSE: 0.1424 | MAE: 0.1079 | R² Score: 0.6465



(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2704) 
(ClientAppActor pid=2704)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2706) 
(ClientAppActor pid=2706)         
(ClientAppActor pid=2706) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 9x across cluster]
(ClientAppActor pid=2706)             This is a deprecated feature. It will be removed [repeated 9x across cluster]
(ClientAppActor pid=27

Federated Learning Championship Run finished.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


#FL without novelty

##Install necessary libraries

In [ ]:
!pip install flwr["simulation"] torch pandas scikit-learn -q flwr

##Code to ignore some deprecation warnings

In [ ]:
# Cell 1 (The very first cell in your notebook)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppresses TensorFlow/XLA warnings

print("TensorFlow logging is configured to be silent.")

TensorFlow logging is configured to be silent.


In [ ]:
# Keep this cell at the top of your notebook (e.g., as Cell 2)
import logging
import warnings

# This silences the SERVER-SIDE INFO logs
logging.getLogger("flwr").setLevel(logging.ERROR)

# This is a good general practice, but the client-side fix is more important
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

print("Logging and warnings configured for a cleaner output.")

Logging and warnings configured for a cleaner output.


##Model Code

In [ ]:
# Replace your existing Cell 4 with this entire block of code

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from collections import OrderedDict
import flwr as fl
import numpy as np
from typing import Dict, List, Tuple
from flwr.common import Context

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# ========== Dataset (with Data Augmentation) - UNCHANGED ==========
class FatigueSessionDataset(Dataset):
    def __init__(self, df, session_map=None, compute_elapsed=True):
        df = df.copy()
        df['session'] = df['session'].astype(int)
        df['person'] = df['person'].astype(int)
        df['window_start'] = pd.to_datetime(df['window_start'])
        if compute_elapsed:
            df['elapsed_time'] = df.groupby(['person', 'session'])['window_start'].transform(lambda x: (x - x.min()).dt.total_seconds())
        else:
            df['elapsed_time'] = 0.0
        self.data = df.sort_values(['person', 'session', 'window_start']).reset_index(drop=True)
        self.session_map = session_map if session_map is not None else {s: i for i, s in enumerate(sorted(df['session'].unique()))}

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx].copy()
        noise_level = 0.015
        row['hr_mean'] += np.random.normal(0, noise_level)
        row['duration_mean'] += np.random.normal(0, noise_level)
        row['ax_mean'] += np.random.normal(0, noise_level)
        row['ay_mean'] += np.random.normal(0, noise_level)
        row['az_mean'] += np.random.normal(0, noise_level)
        row['eda_mean'] += np.random.normal(0, noise_level)
        row['temp_mean'] += np.random.normal(0, noise_level)
        HR = torch.tensor([row['hr_mean'], row['hr_std']], dtype=torch.float32)
        IBI = torch.tensor([row['duration_mean'], row['duration_std']], dtype=torch.float32)
        ACC = torch.tensor([row['ax_mean'], row['ax_std'], row['ay_mean'], row['ay_std'], row['az_mean'], row['az_std']], dtype=torch.float32)
        EDA = torch.tensor([row['eda_mean'], row['eda_std']], dtype=torch.float32)
        Temp = torch.tensor([row['temp_mean'], row['temp_std']], dtype=torch.float32)
        phys = torch.tensor([row['physicalFatigueScore']], dtype=torch.float32)
        ment = torch.tensor([row['mentalFatigueScore']], dtype=torch.float32)
        session_index = self.session_map[int(row['session'])]
        session_feat = torch.tensor([float(session_index)], dtype=torch.float32)
        elapsed = torch.tensor([float(row['elapsed_time'])], dtype=torch.float32)
        domain_disc = torch.tensor([0.0], dtype=torch.float32) # Still needed for data loading signature
        return [HR, IBI, ACC, EDA, Temp], domain_disc, session_feat, elapsed, (phys, ment)

def collate(batch):
    modal_inputs = [torch.stack([b[0][i] for b in batch]) for i in range(5)]
    domain_disc  = torch.stack([b[1] for b in batch])
    session_feat = torch.stack([b[2] for b in batch])
    elapsed_feat = torch.stack([b[3] for b in batch])
    phys         = torch.stack([b[4][0] for b in batch])
    ment         = torch.stack([b[4][1] for b in batch])
    return modal_inputs, domain_disc, session_feat, elapsed_feat, (phys, ment)

# ========== Model Components ==========
class ModalityLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=32):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
    def forward(self, x):
        if x.ndim == 2: x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        return out[:, -1, :]

# --- NOTE: CrossModalAttention and DomainAdaptiveLayer have been REMOVED ---

# ========== NEW Main Model Architecture (Baseline Version) ==========
class BaselineModel_NoFusion(nn.Module):
    def __init__(self, modalities_dim, lstm_hidden=32, num_sessions=3, use_time_and_session=True):
        super().__init__()
        self.use_time_and_session = use_time_and_session
        self.modality_lstms = nn.ModuleList([ModalityLSTM(d, lstm_hidden) for d in modalities_dim])

        # Session and Time embeddings remain the same
        self.session_emb = nn.Embedding(num_sessions, 8)
        self.time_enc = nn.Sequential(nn.Linear(1, 8), nn.ReLU(), nn.Linear(8, 8))

        # --- MODIFIED: The input to the global LSTM is now the concatenation of all modality LSTMs ---
        # 5 modalities * lstm_hidden_size + session_emb_size + time_emb_size
        concatenated_dim = len(modalities_dim) * lstm_hidden
        global_input_dim = concatenated_dim + 16 if use_time_and_session else concatenated_dim

        self.global_lstm = nn.LSTM(global_input_dim, 32, batch_first=True)
        self.reg_phys = nn.Linear(32, 1)
        self.reg_ment = nn.Linear(32, 1)
        self.apply(self._init_weights)

    @staticmethod
    def _init_weights(m):
        if isinstance(m, (nn.Linear)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None: nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LSTM):
            for name, param in m.named_parameters():
                if 'weight' in name: nn.init.xavier_uniform_(param)
                elif 'bias' in name: nn.init.zeros_(param)

    def forward(self, modal_inputs, domain_disc, session_feat=None, elapsed_feat=None, debug=False):
        # 1. Get features from each modality LSTM
        feats = [m(inp) for m, inp in zip(self.modality_lstms, modal_inputs)]

        # 2. NEW: Simple Concatenation instead of Attention Fusion
        fused = torch.cat(feats, dim=1)

        # 3. Add session and time features
        if self.use_time_and_session:
            session_emb = self.session_emb(session_feat.squeeze(-1).long())
            time_emb = self.time_enc(elapsed_feat)
            fused = torch.cat([fused, session_emb, time_emb], dim=1)

        # 4. Global LSTM and regression heads remain the same
        glstm_out, _ = self.global_lstm(fused.unsqueeze(1))
        feat = glstm_out[:, -1, :]
        return self.reg_phys(feat), self.reg_ment(feat)

##Train and Test Function

In [ ]:
# Replace your existing Cell 6 with this
def train(net, trainloader, epochs):
    net.train()
    optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)
    criterion = nn.MSELoss()
    for _ in range(epochs):
        for modals, dom, session_feat, elapsed_feat, (phys, ment) in trainloader:
            modals = [m.to(DEVICE) for m in modals]
            dom, session_feat, elapsed_feat = dom.to(DEVICE), session_feat.to(DEVICE), elapsed_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)
            optimizer.zero_grad()
            p_pred, m_pred = net(modals, dom, session_feat, elapsed_feat)
            loss = criterion(p_pred, phys) + 2 * criterion(m_pred, ment)
            loss.backward()
            optimizer.step()

def test(net, testloader):
    net.eval()
    criterion = nn.MSELoss()
    total_loss, all_phys_preds, all_phys_labels, all_ment_preds, all_ment_labels = 0.0, [], [], [], []
    with torch.no_grad():
        for modals, dom, session_feat, elapsed_feat, (phys, ment) in testloader:
            modals = [m.to(DEVICE) for m in modals]
            dom, session_feat, elapsed_feat = dom.to(DEVICE), session_feat.to(DEVICE), elapsed_feat.to(DEVICE)
            phys, ment = phys.to(DEVICE), ment.to(DEVICE)
            p_pred, m_pred = net(modals, dom, session_feat, elapsed_feat)
            total_loss += (criterion(p_pred, phys) + criterion(m_pred, ment)).item()
            all_phys_preds.extend(p_pred.squeeze().cpu().numpy())
            all_phys_labels.extend(phys.squeeze().cpu().numpy())
            all_ment_preds.extend(m_pred.squeeze().cpu().numpy())
            all_ment_labels.extend(ment.squeeze().cpu().numpy())
    loss = total_loss / len(testloader) if len(testloader) > 0 else 0
    rmse_phys = math.sqrt(mean_squared_error(all_phys_labels, all_phys_preds))
    mae_phys = mean_absolute_error(all_phys_labels, all_phys_preds)
    r2_phys = r2_score(all_phys_labels, all_phys_preds)
    rmse_ment = math.sqrt(mean_squared_error(all_ment_labels, all_ment_preds))
    mae_ment = mean_absolute_error(all_ment_labels, all_ment_preds)
    r2_ment = r2_score(all_ment_labels, all_ment_preds)
    metrics = {"loss": loss, "rmse_physical": rmse_phys, "mae_physical": mae_phys, "r2_physical": r2_phys, "rmse_mental": rmse_ment, "mae_mental": mae_ment, "r2_mental": r2_ment}
    return loss, metrics

##Dataset Processing

In [ ]:
# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Update this path ---
CSV_PATH = '/content/drive/MyDrive/Fatigue_Set/final_feature_label_dataset_normalized_interpolated.csv'
NUM_CLIENTS = 10

def load_datasets():
    """Loads the dataset and partitions it for the clients."""
    df = pd.read_csv(CSV_PATH)
    session_map = {s: i for i, s in enumerate(sorted(df['session'].astype(int).unique()))}
    num_sessions = len(session_map)
    full_dataset = FatigueSessionDataset(df, session_map=session_map, compute_elapsed=True)
    len_train = int(len(full_dataset) * 0.8)
    len_val = len(full_dataset) - len_train
    ds_train, ds_val = random_split(full_dataset, [len_train, len_val], torch.Generator().manual_seed(42))
    partition_size = len(ds_train) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    for i in range(len(ds_train) % NUM_CLIENTS): lengths[i] += 1
    datasets = random_split(ds_train, lengths, torch.Generator().manual_seed(42))
    trainloaders, valloaders = [], []
    for ds in datasets:
        trainloaders.append(DataLoader(ds, batch_size=32, shuffle=True, collate_fn=collate, drop_last=True)) # Bug fix
        valloaders.append(DataLoader(ds_val, batch_size=32, shuffle=False, collate_fn=collate))
    testloader = DataLoader(ds_val, batch_size=32, shuffle=False, collate_fn=collate)
    return trainloaders, valloaders, testloader, num_sessions

trainloaders, valloaders, testloader, NUM_SESSIONS = load_datasets()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Flower Client Code

In [ ]:
# Replace your existing Cell 7 with this
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net, self.trainloader, self.valloader = net, trainloader, valloader
    def get_parameters(self, config): return [val.cpu().numpy() for _, val in self.net.state_dict().items()]
    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)
    def fit(self, parameters, config):
        self.set_parameters(parameters)
        train(self.net, self.trainloader, epochs=8) # Using the simple train for FedAvg
        return self.get_parameters(config={}), len(self.trainloader.dataset), {}
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, metrics = test(self.net, self.valloader)
        return loss, len(self.valloader.dataset), metrics

def client_fn(cid: str) -> fl.client.Client:
    # --- MODIFIED: Create an instance of the new baseline model ---
    net = BaselineModel_NoFusion([2,2,6,2,2], lstm_hidden=32, num_sessions=NUM_SESSIONS).to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(net, trainloader, valloader).to_client()

##Server Strategy Code

In [ ]:
# Replace your existing Cell 8 with this
from flwr.server.strategy import FedAvg
from flwr.common import ndarrays_to_parameters

def get_evaluate_fn(testloader, num_sessions):
    def evaluate(server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]):
        # --- MODIFIED: Create an instance of the new baseline model ---
        net = BaselineModel_NoFusion([2,2,6,2,2], lstm_hidden=32, num_sessions=num_sessions).to(DEVICE)

        params_dict = zip(net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        net.load_state_dict(state_dict, strict=True)
        loss, metrics = test(net, testloader)
        print("\n" + "="*50)
        print(f"Server-side evaluation round {server_round} (Baseline Model)") # Added note for clarity
        print(f"  - Loss: {metrics['loss']:.4f}")
        print(f"  - Physical Fatigue:")
        print(f"    - RMSE: {metrics['rmse_physical']:.4f} | MAE: {metrics['mae_physical']:.4f} | R² Score: {metrics['r2_physical']:.4f}")
        print(f"  - Mental Fatigue:")
        print(f"    - RMSE: {metrics['rmse_mental']:.4f} | MAE: {metrics['mae_mental']:.4f} | R² Score: {metrics['r2_mental']:.4f}")
        print("="*50 + "\n")
        return loss, metrics
    return evaluate

# --- MODIFIED: Create an instance of the new baseline model for initial parameters ---
net = BaselineModel_NoFusion([2,2,6,2,2], lstm_hidden=32, num_sessions=NUM_SESSIONS).to(DEVICE)
initial_parameters = ndarrays_to_parameters([val.cpu().numpy() for _, val in net.state_dict().items()])

strategy = FedAvg(
    fraction_fit=1.0, fraction_evaluate=1.0, min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS, min_available_clients=NUM_CLIENTS,
    evaluate_fn=get_evaluate_fn(testloader, NUM_SESSIONS),
    initial_parameters=initial_parameters,
    on_evaluate_config_fn=lambda r: {"round": r},
)

##Start Simulation

In [ ]:
# Start the simulation for 80 rounds
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=80),
    strategy=strategy,
)

print("Federated Learning Championship Run finished.")

2025-09-07 08:16:00,079	INFO worker.py:1771 -- Started a local Ray instance.



Server-side evaluation round 0 (Baseline Model)
  - Loss: 1.5586
  - Physical Fatigue:
    - RMSE: 0.6314 | MAE: 0.5949 | R² Score: -7.5826
  - Mental Fatigue:
    - RMSE: 1.0786 | MAE: 1.0408 | R² Score: -19.2855



(pid=12028) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=12028) E0000 00:00:1757232980.744085   12028 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=12028) E0000 00:00:1757232980.758005   12028 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=12027) W0000 00:00:1757232980.800407   12027 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(pid=12027) W0000 00:00:1757232980.800471   12027 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(pid=12027) W0000 00:00:1757232980.800479   12027 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid lin


Server-side evaluation round 1 (Baseline Model)
  - Loss: 0.1064
  - Physical Fatigue:
    - RMSE: 0.2114 | MAE: 0.1745 | R² Score: 0.0378
  - Mental Fatigue:
    - RMSE: 0.2490 | MAE: 0.2032 | R² Score: -0.0809



(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12028) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 5x across cluster]
(ClientAppActor pid=12028)             This is a deprecated feature. It will be removed [repeated 5x across cluster]
(ClientAppActor pid=12028)             entirely in future versions of Flower. [repeated 5x across cluster]
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=120

...


Server-side evaluation round 79 (Baseline Model)
  - Loss: 0.0789
  - Physical Fatigue:
    - RMSE: 0.1891 | MAE: 0.1494 | R² Score: 0.2299
  - Mental Fatigue:
    - RMSE: 0.2079 | MAE: 0.1662 | R² Score: 0.2467



(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12027) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 11x across cluster]
(ClientAppActor pid=12027)             This is a depre


Server-side evaluation round 80 (Baseline Model)
  - Loss: 0.0791
  - Physical Fatigue:
    - RMSE: 0.1889 | MAE: 0.1492 | R² Score: 0.2317
  - Mental Fatigue:
    - RMSE: 0.2082 | MAE: 0.1672 | R² Score: 0.2442



(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12028) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 6x across cluster]
(ClientAppActor pid=12028)             This is a deprecated feature. It will be removed [repeated 6x across cluster]
(ClientAppActor pid=12028)             entirely in future versions of Flower. [repeated 6x across cluster]
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=12027) 
(ClientAppActor pid=12027)         
(ClientAppActor pid=12028) 
(ClientAppActor pid=12028)         
(ClientAppActor pid=120

Federated Learning Championship Run finished.
